In [ ]:
#7.1.1 CommaSeparatedListOutputParser
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="Suggest some names for my {subject} startup.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

model = OpenAI(temperature=0)

_input = prompt.format(subject="Mobile")
output = model(_input)

output_parser.parse(output)

['MobilityX',
 'MobileMakers',
 'MobileMania',
 'MobileMoguls',
 'MobileMasters',
 'MobileMeisters',
 'MobileMavens',
 'MobileManiacs',
 'MobileMentors',
 'MobileMasters',
 'MobileMoguls',
 'MobileManiacs',
 'MobileMania',
 'MobileMentors',
 'MobileMavens.']

In [ ]:
from langchain.output_parsers import  CommaSeparatedListOutputParser

prompt = PromptTemplate.from_template(
    "Answer the following question: {question}"
)
parser = CommaSeparatedListOutputParser()
model = OpenAI(temperature=0)
chain = prompt | model | parser

list(chain.invoke({"question": "Suggest some names for baby"}))

['girl\n\n1. Abigail\n2. Amelia\n3. Ava\n4. Bella\n5. Chloe\n6. Daisy\n7. Ella\n8. Emma\n9. Freya\n10. Grace']

In [ ]:
#7.1.2 Custom OutputParser

from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import OutputFixingParser

response_schemas = [
    ResponseSchema(name="Monument", description="The monument mentioned in the answer."),
    ResponseSchema(name="city", description="the city in which the monument is stored."),
    ResponseSchema(name="architect", description="the architect of the monument."),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

model = OpenAI(temperature=0)

_input = prompt.format_prompt(question="Give example of a monument in North America with details about them")
output = model(_input.to_string())
output_parser.parse(output)

{'Monument': 'Statue of Liberty', 'city': 'New York City', 'architect': 'Frédéric Auguste Bartholdi'}


In [ ]:
#7.1.3 Magic Output Fixer

from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from typing import List

class state(BaseModel):
    state: str = Field(description="name of a indian state")
    cities: List[str] = Field(description="Indian cities in that state")

parser = PydanticOutputParser(pydantic_object=state)

format_instructions = parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

model = OpenAI(temperature=0)
query = "Choose a random Indian state and pen down its some cities"

_input = prompt.format_prompt(question=query)
output = model(_input.to_string())
try:
  print(parser.parse(output))
except:
  new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())
  print(new_parser.parse(output))

state='Karnataka' cities=['Bengaluru', 'Mysore', 'Hubli', 'Belgaum', 'Gulbarga']


In [ ]:
#7.2.1 ConversationalBufferMemory

from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import os

os.environ['OPENAI_API_KEY']=''

template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm = OpenAI()
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

llm_chain.predict(human_input="Hi there my friend")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.


Human: Hi there my friend
Chatbot:

> Finished chain.


' Hi there! How can I help you?'

In [ ]:
llm_chain.predict(human_input="Tell me what is Machine Learning?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: Hi there my friend
AI:  Hi there! How can I help you today?
Human: Tell me what is Machine Learning?
Chatbot:

> Finished chain.


' Machine Learning is a type of artificial intelligence that enables computers to learn from data, identify patterns and make decisions without being explicitly programmed to do so.'

In [ ]:
llm_chain.predict(human_input="Anything else you wish to add?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: Hi there my friend
AI:  Hi there! How can I help you today?
Human: Tell me what is Machine Learning?
AI:  Machine Learning is a type of artificial intelligence that enables computers to learn from data, identify patterns and make decisions without being explicitly programmed to do so.
Human: Anything else you wish to add?
Chatbot:

> Finished chain.


' Yes, machine learning algorithms can be used for a variety of tasks, such as recognizing faces, understanding natural language, and making predictions based on data.'

In [ ]:
memory.load_memory_variables({})

{'chat_history': 'Human: Hi there my friend\nAI:  Hi there! How can I help you today?\nHuman: Tell me what is Machine Learning?\nAI:  Machine Learning is a type of artificial intelligence that enables computers to learn from data, identify patterns and make decisions without being explicitly programmed to do so.\nHuman: Anything else you wish to add?\nAI:  Yes, machine learning algorithms can be used for a variety of tasks, such as recognizing faces, understanding natural language, and making predictions based on data.'}

In [ ]:
#7.2.2 ConversationSummaryMemory

from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.llms import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "hi"}, {"output": "whats up"})

llm = OpenAI(temperature=0)
chat = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [ ]:
chat.predict(input='Hey dude !! do you what happened to Nandu?')
chat.predict(input='He met with a car accident yesterday. Suggest me some hospitals for a good treatment')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI, to which the AI responds.
Human: Hey dude !! do you what happened to Nandu?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI, to which the AI responds. The human then asks about Nandu, to which the AI responds that they don't know who Nandu is and asks for more information.
Human: He met with a car accident yesterday. Suggest m

" I'm sorry to hear that Nandu was in a car accident. I'm not familiar with hospitals in your area, but I can provide you with some resources to help you find the best hospital for Nandu's treatment. Would that be helpful?"

In [ ]:
chat.predict(input='What should we do next in such a situation?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human greets the AI, to which the AI responds. The human then asks about Nandu, to which the AI responds that they don't know who Nandu is and asks for more information. The human then informs the AI that Nandu had a car accident, and the AI offers to provide resources to help the human find the best hospital for Nandu's treatment.
Human: What should we do next in such a situation?
AI:

> Finished chain.


' It depends on the severity of the accident and the injuries sustained. If the injuries are serious, it is best to seek medical attention as soon as possible. I can provide you with a list of nearby hospitals that have the best medical care and resources available.'

In [ ]:
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

'\nThe human greets the AI and asks about a person named Nandu who had a car accident. The AI is not familiar with Nandu, but offers to provide resources to help the human find the best hospital for treatment. The AI then suggests that, depending on the severity of the accident, it is best to seek medical attention as soon as possible and offers to provide a list of nearby hospitals with the best medical care and resources.'